In [1]:
import pandas as pd, numpy as np
import glob
from datetime import datetime, date

## Input all files

In [3]:
path = r'./player_stats_table' # use your path
files_path = glob.glob(path + "/*.csv")

mutiple_files = [pd.read_csv(file, index_col = None) for file in files_path]
df = pd.concat(mutiple_files, axis  = 0, ignore_index=True)
df.drop(df.columns[0], axis = 1, inplace = True)
df.head(5)

,Date,Game_Index,Home_Away,Team,Players,MP,FG,FGA,FG%,3P,...,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg
0,2018-10-16,1,Home,BOS,Al Horford,29:57,4.0,7.0,0.571,0.0,...,0.0,13.1,6.3,9.0,0.0,10.5,28.7,14.3,90.0,80.0
1,2018-10-16,1,Home,BOS,Kyrie Irving,29:00,2.0,14.0,0.143,1.0,...,0.0,13.5,6.5,29.9,0.0,0.0,16.8,25.3,63.0,89.0
2,2018-10-16,1,Home,BOS,Jayson Tatum,28:56,9.0,17.0,0.529,1.0,...,6.3,23.7,14.6,18.4,1.6,0.0,5.0,28.6,125.0,83.0
3,2018-10-16,1,Home,BOS,Jaylen Brown,28:23,5.0,13.0,0.385,1.0,...,9.6,6.9,8.3,10.1,0.0,0.0,12.6,22.9,91.0,89.0
4,2018-10-16,1,Home,BOS,Gordon Hayward,24:37,4.0,12.0,0.333,1.0,...,7.4,11.9,9.6,0.0,7.5,0.0,0.0,21.4,90.0,74.0


### Teams Played by Date

> The following information is used to scrape line-up data.

In [4]:
team_played_date = df.loc[:, ["Date", "Game_Index", "Team"]].drop_duplicates()
team_played_date.reset_index(drop = True, inplace = True)
date_str = team_played_date["Date"].map(str)

def getGameYear(date_col):
    gameYear = []
    for day in date_str:
        if int(day.split("-")[1]) >= 9:
            gameYear.append(day.split("-")[0][-2:])
        else:
            gameYear.append(str(int(day.split("-")[0][-2:]) - 1))
    return gameYear

team_played_date["GameYear"] = getGameYear(date_str)

In [5]:
#team_played_date.to_csv("Team_Played_Date.csv", index = False)

## Summary

In [15]:
df.shape

(323024, 39)

In [52]:
# Missing Value
col_has_missing = df.isnull().apply(sum, axis = 0) > 0
pd.DataFrame(df.isnull().apply(sum, axis = 0)[col_has_missing]).transpose()

,FG%,3P%,FT%,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg
0,16229,122491,135441,36,14103,16229,16229,16229,59,59,59,63,59,59,11211,59,59,56


## Win or Loss

In [6]:
def SumPoints(group):
    return group["PTS"].sum()


df_drop = df.drop(["Home_Away", "Players", "MP"], axis = 1)
df_drop_grouped = df_drop.groupby(["Date", "Game_Index", "Team"])
pd.DataFrame(df_drop_grouped.apply(SumPoints))

(31042, 1)

## Team Data Transformation

In [113]:
# Minute Played to Second Played
df.insert(6,
          "SP",
          df["MP"].map(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
         )

In [114]:
def team_table_transformed(df):
    df_drop = df.drop(["Players", "MP"], axis = 1)
    df_drop_grouped = df_drop.groupby(["Date", "Game_Index", "Team", "Home_Away"])

    def weightedavg(group, col):
        d = group[col]
        w = group["SP"]
        return (d * w).sum() / w.sum()

    table_transformed = []
    for col in list(df_drop.columns[4:]):
        table_transformed.append(df_drop_grouped.apply(lambda x: weightedavg(x, col)))
    new_df = pd.concat(table_transformed, axis = 1)
    new_df.columns = list(short_df.columns[4:])
    return new_df

In [ ]:
short_df = df.iloc[:100, :]
df_drop = df.drop(["Players", "MP"], axis = 1)
df_drop_grouped = df_drop.groupby(["Date", "Game_Index", "Team", "Home_Away"])

def weightedavg(group, col):
    d = group[col]
    w = group["SP"]
    return (d * w).sum() / w.sum()

'''
table_transformed = []
for col in list(df_drop.columns[4:]):
    table_transformed.append(df_drop_grouped.apply(lambda x: weightedavg(x, col)))
new_df = pd.concat(table_transformed, axis = 1)
new_df.columns = list(short_df.columns[4:])
'''


In [115]:
new_df = team_table_transformed(df.iloc[:1000, :])

ValueError: Length mismatch: Expected axis has 34 elements, new values have 33 elements

In [98]:
df_tojoin = df.loc[:, ["Date", "Game_Index", "Team", "Home_Away"]].iloc[:1000, :]

In [102]:
new_df

FG        FGA       FG%        3P       3PA  \
Date       Game_Index Team                                                      
2018-10-16 1          BOS   4.430069  10.494097  0.426938  1.112292  3.913125   
                      PHI   4.645972  11.447917  0.397512  0.686458  3.184583   
           2          GSW   5.278403  12.197847  0.425167  0.963611  3.569514   
                      OKC   4.161389  11.159861  0.331572  1.302917  4.503681   
2018-10-17 1          CHO   5.064028  11.281667  0.444945  2.188542  4.935625   
                      MIL   5.300069  10.725347  0.490787  1.649653  4.147778   
           2          BRK   5.092847  10.602431  0.425361  0.554236  3.393889   
                      DET   4.911458  11.442500  0.416279  0.769236  2.967431   
           3          HOU   4.769028  10.813333  0.420256  2.047222  5.675833   
                      NOP   7.122222  13.316389  0.465026  1.136667  3.212361   
           4          IND   4.903889   8.424792  0.552995  0.995278  2.472431   
                      MEM   2.450000   8.652083  0.292489  1.030347  3.006319   
           5          DEN   3.808264  10.047986  0.380313  0.886042  2.791319   
                      LAC   3.988125   9.433958  0.431718  1.105833  3.327778   
           6          ATL   4.035069   9.062431  0.448035  1.114931  3.749236   
                      NYK   4.396458  10.089792  0.430281  1.185139  3.408611   
           7          MIA   4.785665  12.032435  0.385904  1.157800  4.155438   
                      ORL   4.633143  11.051535  0.424666  0.910821  2.954716   
           8          DAL   4.075000   9.610694  0.467106  0.977361  3.460694   
                      PHO   5.841806  10.520903  0.487601  2.538819  4.481389   
           9          SAC   6.195278  11.737708  0.508288  0.705347  2.196736   
                      UTA   5.312917   9.781181  0.520647  1.649861  3.521111   
           10         MIN   4.927361  11.309653  0.425191  0.799792  2.498333   
                      SAS   4.993403  12.035625  0.420546  1.130069  2.876250   
           11         CLE   3.784653   9.701875  0.422062  0.834653  2.100417   
                      TOR   5.333472  10.885347  0.507488  1.703125  3.738819   
2018-10-18 1          CHI   5.314931  11.706875  0.424055  1.381875  4.051944   
                      PHI   5.090972  10.752153  0.480041  1.442361  4.115486   
           2          LAL   5.377500  10.968750  0.477430  0.790903  3.403819   
                      POR   4.693681  10.797292  0.446380  1.443819  4.064444   
...                              ...        ...       ...       ...       ...   
2018-10-20 7          NYK   4.214028   9.850903  0.435317  1.455486  3.950625   
           8          ORL   5.539306  11.413819  0.412441  2.118472  3.666667   
                      PHI   5.548958  11.816042  0.458225  2.453819  4.619583   
           9          POR   4.530347   8.278542  0.529535  1.548750  3.161181   
                      SAS   4.792153  10.836458  0.393074  1.265139  2.888750   
           10         TOR   4.377778   9.981181  0.425518  1.269444  3.554444   
                      WAS   4.582778  10.259028  0.385795  1.576458  4.524722   
2018-10-21 1          ATL   5.862083  11.483264  0.479434  2.552500  5.585694   
                      CLE   5.020347  11.357500  0.494292  1.155278  3.375486   
           2          DEN   4.012086   9.697923  0.390940  0.685490  3.666458   
                      GSW   4.852817  10.933042  0.496146  1.131972  4.398347   
           3          HOU   5.533194  11.481597  0.488728  1.982986  4.498958   
                      LAC   4.539306   9.894514  0.504678  0.537431  2.910764   
           4          OKC   5.503125  12.138819  0.442576  0.993125  4.163542   
                      SAC   5.677500   9.829444  0.585255  1.239375  2.621458   
2018-10-22 1          BOS   4.117708  10.194722  0.400407  1.004931  4.516111   
                      ORL   4.711875  11.121806  0.376044  1.3188

In [101]:
pd.merge(left = new_df, right = df_tojoin, on = ["Date", "Game_Index", "Team"])

,Date,Game_Index,Team,FG,FGA,FG%,3P,3PA,3P%,FT,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,Home_Away
0,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
1,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
2,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
3,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
4,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
5,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
6,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
7,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
8,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
9,2018-10-16,1,BOS,4.430069,10.494097,0.426938,1.112292,3.913125,0.284612,1.173750,...,17.478222,10.789229,12.069382,1.346535,1.641771,13.402563,20.002868,97.576389,83.626111,Home
